In [2]:
import torch
import torch.nn.functional as F

In [14]:
z1 = torch.randn(3, 5)
z2 = torch.randn(3, 5)

z1_n0 = F.normalize(z1, dim=0)
z2_n0 = F.normalize(z2, dim=0)

z1_n1 = F.normalize(z1, dim=1)
z2_n1 = F.normalize(z2, dim=1)

print(z1)
print(z1_n0)
print(z1_n1)

tensor([[ 0.3015,  0.7135, -1.7389,  1.6651,  2.3736],
        [-1.8491, -1.1479,  0.5863, -0.5880,  1.9614],
        [ 0.2790,  0.9023, -0.2384,  1.0261,  1.0308]])
tensor([[ 0.1592,  0.4390, -0.9397,  0.8153,  0.7310],
        [-0.9762, -0.7064,  0.3168, -0.2879,  0.6041],
        [ 0.1473,  0.5552, -0.1288,  0.5024,  0.3175]])
tensor([[ 0.0869,  0.2057, -0.5013,  0.4801,  0.6843],
        [-0.6072, -0.3770,  0.1925, -0.1931,  0.6441],
        [ 0.1594,  0.5154, -0.1362,  0.5862,  0.5889]])


In [21]:
print(z1.T @ z2)
print(z1_n0.T @ z2_n0)
print(z1_n1.T @ z2_n1)

c = z1_n0.T @ z2_n0 / 3

tensor([[ 1.3341,  0.7255,  1.7563, -1.8413,  0.3239],
        [ 0.9806,  0.3093,  2.6667, -1.2414, -0.4104],
        [-2.0712, -0.1347, -4.6917,  1.3817, -0.2528],
        [ 1.5604,  0.0108,  4.8011, -1.1641, -0.5229],
        [ 0.6905, -1.1013,  5.3146,  0.8950, -1.1448]])
tensor([[ 0.5229,  0.8400,  0.3626, -0.8842,  0.1665],
        [ 0.4481,  0.4175,  0.6417, -0.6948, -0.2458],
        [-0.8311, -0.1596, -0.9915,  0.6791, -0.1330],
        [ 0.5673,  0.0116,  0.9193, -0.5184, -0.2492],
        [ 0.1579, -0.7439,  0.6401,  0.2507, -0.3431]])
tensor([[ 0.2467,  0.1689,  0.3485, -0.4142,  0.0099],
        [ 0.0212,  0.0465,  0.4810, -0.2055, -0.3179],
        [-0.2206, -0.0346, -0.5644,  0.2022,  0.0346],
        [ 0.0073, -0.0254,  0.6834, -0.1127, -0.3967],
        [-0.3102, -0.2932,  0.5637,  0.4412, -0.5676]])


In [18]:
def off_diagonal(x):
    # return a flattened view of the off-diagonal elements of a square matrix
    n, m = x.shape
    assert n == m
    return x.flatten()[:-1].view(n - 1, n + 1)[:, 1:].flatten()

In [22]:
invariance = torch.diagonal(c).add_(-1).pow_(2).sum()
redundancy_reduction = off_diagonal(c).pow_(2).sum()
loss = invariance + redundancy_reduction
print(loss)

tensor(6.5079)


In [24]:
print(c)
c.div_(2)
print(c)

tensor([[ 0.6818,  0.2800,  0.1209, -0.2947,  0.0555],
        [ 0.1494,  0.7410,  0.2139, -0.2316, -0.0819],
        [-0.2770, -0.0532,  1.7702,  0.2264, -0.0443],
        [ 0.1891,  0.0039,  0.3064,  1.3755, -0.0831],
        [ 0.0526, -0.2480,  0.2134,  0.0836,  1.2418]])
tensor([[ 0.3409,  0.1400,  0.0604, -0.1474,  0.0277],
        [ 0.0747,  0.3705,  0.1070, -0.1158, -0.0410],
        [-0.1385, -0.0266,  0.8851,  0.1132, -0.0222],
        [ 0.0945,  0.0019,  0.1532,  0.6877, -0.0415],
        [ 0.0263, -0.1240,  0.1067,  0.0418,  0.6209]])


### Export model

In [4]:
import torch
from nets import SiameseNetSync
from torchvision.models import resnet18

In [5]:
input_data = torch.rand(1,3,32,32)
model = SiameseNetSync(resnet18())
model(input_data)

TypeError: forward() missing 1 required positional argument: 'x2'

In [2]:
input_tensor = torch.rand((1, 1, 128, 128), dtype=torch.float32)

model = MyModel()

torch.onnx.export(
    model,                  # model to export
    (input_tensor,),        # inputs of the model,
    "my_model.onnx",        # filename of the ONNX model
    input_names=["input"],  # Rename inputs for the ONNX model
    dynamo=True             # True or False to select the exporter to use
)

c:\Users\Utente\anaconda3\envs\torch-wb2\lib\site-packages\onnxscript\converter.py:820: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
c:\Users\Utente\anaconda3\envs\torch-wb2\lib\site-packages\onnxscript\converter.py:820: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()


[torch.onnx] Obtain model graph for `MyModel([...]` with `torch.export.export`...
[torch.onnx] Obtain model graph for `MyModel([...]` with `torch.export.export`... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅


ONNXProgram(
    model=
        <
            ir_version=9,
            opset_imports={'': 18, 'pkg.onnxscript.torch_lib.common': 1},
            producer_name='pytorch',
            producer_version='2.5.1',
            domain=None,
            model_version=None,
        >
        graph(
            name=main_graph,
            inputs=(
                %"input"<FLOAT,[1,1,128,128]>
            ),
            outputs=(
                %"relu"<FLOAT,[1,128,124,124]>
            ),
            initializers=(
                %"conv1.weight"<FLOAT,[128,1,5,5]>,
                %"conv1.bias"<FLOAT,[128]>
            ),
        ) {
            0 |  # node_Conv_0
                 %"convolution"<FLOAT,[1,128,124,124]> ⬅️ ::Conv(%"input", %"conv1.weight", %"conv1.bias") {auto_pad=NOTSET, dilations=[1, 1], group=1, pads=[0, 0, 0, 0], strides=[1, 1]}
            1 |  # node_Relu_1
                 %"relu"<FLOAT,[1,128,124,124]> ⬅️ ::Relu(%"convolution")
            return %"relu"<FLOAT,[1,128,12